In [ ]:
from feed import Feed, Loom
import geopandas as gpd
import pandas as pd

folder='/home/gamma/gtfs-feed-vis/uud/gtfs'

agency=pd.read_csv(folder+'/agency.txt', sep=',')
calendar=pd.read_csv(folder+'/calendar.txt', sep=',')
calendar_dates=pd.read_csv(folder+'/calendar_dates.txt', sep=',')
routes=pd.read_csv(folder+'/routes.txt', sep=',')
trips=pd.read_csv(folder+'/trips.txt', sep=',')
stops=pd.read_csv(folder+'/stops.txt', sep=',')
stop_times=pd.read_csv(folder+'/stop_times.txt', sep=',')
shapes=pd.read_csv(folder+'/shapes.txt', sep=',')
shapes

In [ ]:
stop_id=12
stop=stops[stops.stop_id==stop_id]
stop

In [ ]:
feed=Feed(folder)

In [ ]:
stop=feed.DescribeForFlag(stop_id)

In [ ]:
stops_selection, stop_times_selection, trips_selection, shape_selection, routes_selection=feed.GetGeomOfStop(stop_id)
shape_selection

In [ ]:
routes_selection

In [ ]:
shape_selection[shape_selection.shape_id=='t4_dir1_shape'].plot()

In [ ]:
from matplotlib.path import Path
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry

import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM

def GetExtentOfShapes(shapes):
    extent=shapely.geometry.box(*shapes.total_bounds)
    extent_buff=extent.buffer(0.005).bounds
    return [extent_buff[0], extent_buff[2], extent_buff[1], extent_buff[3]]

imagery = OSM()

theta = np.linspace(0, 2 * np.pi, 100)
circle_verts = np.vstack([np.sin(theta), np.cos(theta)]).T
concentric_circle = Path.make_compound_path(Path(circle_verts[::-1]),
                                            Path(circle_verts * 0.6))

fig = plt.figure(figsize=[16,12])
ax = fig.add_subplot(1, 1, 1, projection=imagery.crs)
extent=GetExtentOfShapes(feed.shapes)

#print(extent)
ax.set_extent(list(extent), ccrs.Geodetic())
ax.add_image(imagery, 7)
shape_ids=list(set(shape_selection.shape_id))
for shape_id in shape_ids:
    print(shape_id)
    shape_selection_plotter=shape_selection[shape_selection.shape_id==shape_id]
    xs, ys = list(shape_selection_plotter.shape_pt_lon), list(shape_selection_plotter.shape_pt_lat)
    ax.plot(xs, ys, linestyle='-', linewidth=10, transform=ccrs.Geodetic())
    #ax.add_geometries(shape_selection.geometry, crs=shape_selection.crs)
plt.show()

In [ ]:
extent=feed.shapes.total_bounds
list(extent)

In [ ]:
print(imagery.crs)

In [1]:
from feed import Feed, Loom
import geopandas as gpd
import pandas as pd

loom=Loom('/home/gamma/gtfs-feed-vis/uud/gtfs', stop_id=23)
loom.gtfs2graph()


/home/gamma/gtfs-feed-vis/feed.py:177: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  shp_empty=pd.concat([shp_empty, shp_serving])


In [2]:
loom.topo(max_aggr_dist=250, smooth=0.1)

In [3]:
loom.loom(optim_method='ilp', ilp=True, ilp_num_threads=10, ilp_solver='cbc')

In [4]:
loom.transitmap(render_engine='svg', line_width=40, line_spacing=10, render_dir_markers=True, tight_stations=True, no_render_node_connections=True)

In [5]:
loom.transitmap(render_engine='mvt', z=[14,15,16,17], line_width=40, line_spacing=10, render_dir_markers=True, tight_stations=True, no_render_node_connections=True)

In [17]:
import mapbox_vector_tile
import os
import math

def pixel2deg(xtile, ytile, zoom, xpixel, ypixel, extent = 4096):
    # from https://gis.stackexchange.com/a/460173/44746
    n = 2.0 ** zoom
    xtile = xtile + (xpixel / extent)
    ytile = ytile + ((extent - ypixel) / extent)
    lon_deg = (xtile / n) * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lon_deg, lat_deg)

inner_connections=[]
lines=[]
stations=[]
zoomfolder='/home/gamma/gtfs-feed-vis/uud/gtfs/stop_23/14'
for subdir, dirs, files in os.walk(zoomfolder):
    for file in files:
        #print(os.path.join(subdir, file))
        tile=os.path.join(subdir, file)
        with open(tile, 'rb') as f:
            data=f.read()
        tile_zoom=int(tile.split('/')[-3])
        tile_x=int(tile.split('/')[-2])
        tile_y=int(tile.split('/')[-1][:-4])
        decoded_data = mapbox_vector_tile.decode(data, transformer = lambda x, y: pixel2deg(tile_x, tile_y, tile_zoom, x, y))
        inner_connections.append(decoded_data['inner-connections'])
        lines.append(decoded_data['lines'])
        stations.append(decoded_data['stations'])
stations

/tmp/ipykernel_12851/2038492547.py:28: DeprecationWarning: `decode` signature has changed, use `default_options` instead
  decoded_data = mapbox_vector_tile.decode(data, transformer = lambda x, y: pixel2deg(tile_x, tile_y, tile_zoom, x, y))


[{'extent': 1024,
  'version': 2,
  'features': [{'geometry': {'type': 'Polygon',
     'coordinates': [[[107.58037805557251, 51.83576757614712],
       [107.58039951324463, 51.83589353718612],
       [107.58046388626099, 51.83601618312236],
       [107.58056044578552, 51.83612888449898],
       [107.58069455623627, 51.83622832665588],
       [107.58261501789093, 51.837418300761904],
       [107.58277595043182, 51.837501166868236],
       [107.58296370506287, 51.837560830370364],
       [107.58316218852997, 51.83759729136053],
       [107.58337140083313, 51.837610549895075],
       [107.58357524871826, 51.83760060599454],
       [107.5837790966034, 51.83756414500706],
       [107.5839614868164, 51.83750448150932],
       [107.5841224193573, 51.837421615409085],
       [107.58425652980804, 51.837322175887486],
       [107.58435845375061, 51.83720616283474],
       [107.58441746234894, 51.83708352013967],
       [107.58443892002106, 51.836954247748125],
       [107.58442282676697, 51.8368

In [11]:
with open('/home/gamma/gtfs-feed-vis/uud/gtfs/stop_23/14/13087/5422.mvt', 'rb') as f:
    data = f.read()
decoded_data = mapbox_vector_tile.decode(data)
decoded_data.keys()

dict_keys(['inner-connections', 'lines', 'stations'])